TP2 : Méthodes GMRES et Gradient conjugué

In [1]:
import numpy as np 
import scipy as sp
import scipy.sparse as spsp
import scipy.sparse.linalg as spsplin

In [2]:
def Arnoldi(A, V, H):

    n, p = np.shape(V)         
    Vp = np.zeros((n, p+1))
    Hp = np.zeros((p+1, p))

    Hp[:-1, :-1] = H[:, :]          
    Vp[:, :-1] = V[:, :]


    Av = A@V[:, -1]

    for i in range(p):       
        Hp[i, -1] = Vp[:, i] @ Av

    wp = Av - V @ Hp[:-1, -1]          #<--------------- a modifier
    
    Hp[-1, -1] = np.linalg.norm(wp)
    Vp[:, -1] = wp/Hp[-1, -1]

    return Vp, Hp

In [3]:
def gmres(A, b, xexact):
    n = len(b)
    x0 = np.zeros(n)           

    res0 = b - A @ x0
    resnorm0 = np.linalg.norm(res0)

    V = np.zeros((n, 1))
    V[:, 0] = res0 / resnorm0
    H = np.zeros((1, 0))

    err_list = []
    resnorm_list = []          

    resnorm = 1.
    it = 0
    while (resnorm >= 1e-6) and (it <= n):    
        it += 1

        V, H = Arnoldi(A, V, H)
        Q, R = sp.linalg.qr(H)
        y = np.linalg.solve(R[:-1, :], resnorm0 * Q[0, :-1])
        x = x0 + V[:, :-1] @ y

        resnorm = np.linalg.norm(b - A @ x) / resnorm0
        err = np.linalg.norm(x - xexact) / np.linalg.norm(xexact)

        err_list.append(err)
        resnorm_list.append(resnorm)


    return x, err_list, resnorm_list


In [4]:
#Test :
n = 5

A = np.diag(2*np.ones(n)) + 0.5 * np.random.rand(n, n)/np.sqrt(n)
b = np.ones(n)
x_exact = np.linalg.solve(A, b)

x, err_list, resnorm_list = gmres(A, b, x_exact)

print("Solution : ", x)
print("Erreur : ", err_list)
print("Résidu : ", resnorm_list)


Solution :  [0.36672478 0.36745602 0.35008632 0.42723813 0.4254077 ]
Erreur :  [0.08335044395729141, 0.005579084901842266, 0.00013424740270171986, 1.0598309874494114e-08]
Résidu :  [0.06326169713825255, 0.004365553455550372, 0.00010749052445908867, 8.348025941605074e-09]
